# (Optional) Colab Setup
If you aren't using Colab, you can delete the following code cell. This is just to help students with mounting to Google Drive to access the other .py files and downloading the data, which is a little trickier on Colab than on your local machine using Jupyter. 

In [45]:
# you will be prompted with a window asking to grant permissions
# from google.colab import drive
# drive.mount("/content/drive")

In [46]:
# fill in the path in your Google Drive in the string below. Note: do not escape slashes or spaces
# import os
# datadir = "/content/assignment1"
# if not os.path.exists(datadir):
#   !ln -s "/content/drive/My Drive/YOUR PATH HERE/assignment1/" $datadir
# os.chdir(datadir)
# !pwd

In [47]:
# downloading Fashion-MNIST
# import os
# os.chdir(os.path.join(datadir,"fashion-mnist/"))
# !chmod +x ./get_data.sh
# !./get_data.sh
# os.chdir(datadir)

# Imports

In [48]:
import random
import numpy as np
from data_process import get_FASHION_data, get_RICE_data
from scipy.spatial import distance
from models import Perceptron, SVM, Softmax, Logistic
from kaggle_submission import output_submission_csv
%matplotlib inline
np.random.seed(441)
# For auto-reloading external modules
# See http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Loading Fashion-MNIST

In the following cells we determine the number of images for each split and load the images.
<br /> 
TRAIN_IMAGES + VAL_IMAGES = (0, 60000]
, TEST_IMAGES = 10000

In [105]:
# You can change these numbers for experimentation
# For submission we will use the default values 
TRAIN_IMAGES = 50000
VAL_IMAGES = 10000
normalize = True

In [106]:
data = get_FASHION_data(TRAIN_IMAGES, VAL_IMAGES, normalize=normalize)
X_train_fashion, y_train_fashion = data['X_train'], data['y_train']
X_val_fashion, y_val_fashion = data['X_val'], data['y_val']
X_test_fashion, y_test_fashion = data['X_test'], data['y_test']
n_class_fashion = len(np.unique(y_test_fashion))

# Loading Rice

In [51]:
# loads train / test / val splits of 80%, 20%, 20% 
data = get_RICE_data()
X_train_RICE, y_train_RICE = data['X_train'], data['y_train']
X_val_RICE, y_val_RICE = data['X_val'], data['y_val']
X_test_RICE, y_test_RICE = data['X_test'], data['y_test']
n_class_RICE = len(np.unique(y_test_RICE))

print("Number of train samples: ", X_train_RICE.shape[0])
print("Number of val samples: ", X_val_RICE.shape[0])
print("Number of test samples: ", X_test_RICE.shape[0])

Number of train samples:  10911
Number of val samples:  3637
Number of test samples:  3637


### Get Accuracy

This function computes how well your model performs using accuracy as a metric.

In [52]:
def get_acc(pred, y_test):
    return np.sum(y_test == pred) / len(y_test) * 100

In [76]:
# Hyperparameter search
from itertools import product
from collections import namedtuple
from tqdm.notebook import tqdm
example_params = {
    "model": None,
    "xtrain": None,
    "ytrain": None,
    "xval": None,
    "yval": None,
    "hyperparams": {
        "lr": [],
        "epochs": [],
        "reg_const": []
    }
}
product_dict = lambda dicts: list(dict(zip(dicts, x)) for x in product(*dicts.values()))
def search(params):
    combs = product_dict(params["hyperparams"])
    record = []
    for i, param in enumerate(tqdm(combs)):
        verbose = ' | '.join([(k + "=" + str(v)) for k,v in param.items()])
        print(f"searching {i + 1} / {len(combs)},  { verbose }")
        model = params["model"](**param)
        model.train(params["xtrain"], params["ytrain"])
        val_acc = get_acc(model.predict(params["xval"]), params["yval"])
        print(f"validation accuracy: {val_acc}")
        record.append((verbose, val_acc))
    print(f"DONE. Best parameter is {max(record, key=lambda x: x[1])[0]}")
        

# Perceptron

Perceptron has 2 hyperparameters that you can experiment with:
### Learning rate
The learning rate controls how much we change the current weights of the classifier during each update. We set it at a default value of 0.5, but you should experiment with different values. Here is a guide to help you find a right learning rate: 
- Try values ranging from 5.0 to 0.0005 to see the impact on model accuracy. 
- If the accuracy fluctuates a lot or diverges, the learning rate is too high. Try decreasing it by a factor of 10 (e.g. from 0.5 to 0.05). 
- If the accuracy is changing very slowly, the learning rate may be too low. Try increasing it by a factor of 10.
- You can also try adding a learning rate decay to slowly reduce the learning rate over each training epoch. For example, multiply the learning rate by 0.95 after each epoch.
- Plot training and validation accuracy over epochs for different learning rates. This will help you visualize the impact of the learning rate.
- [Here](https://towardsdatascience.com/https-medium-com-dashingaditya-rakhecha-understanding-learning-rate-dd5da26bb6de) is a detailed guide to learning rate.

### Number of Epochs
An epoch is a complete iterative pass over all of the data in the dataset. During an epoch we predict a label using the classifier and then update the weights of the classifier according to the perceptron update rule for each sample in the training set. You should try different values for the number of training epochs and report your results.

You will implement the Perceptron classifier in the **models/perceptron.py**

The following code: 
- Creates an instance of the Perceptron classifier class 
- The train function of the Perceptron class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy


## Train Perceptron on Fashion-MNIST

In [161]:
params = {
    "model": Perceptron,
    "xtrain": X_train_fashion,
    "ytrain": y_train_fashion,
    "xval": X_test_fashion,
    "yval": y_test_fashion,
    "hyperparams": {
        "n_class": [n_class_fashion],
        "lr": [0.5, 0.1, 0.08, 0.05, 0.01, 0.005],
        "epochs": [10, 15],
    }
}
# search(params)

  0%|          | 0/12 [00:00<?, ?it/s]

searching 1 / 12,  n_class=10 | lr=0.5 | epochs=10


  0%|          | 0/10 [00:00<?, ?it/s]

validation accuracy: 80.93
searching 2 / 12,  n_class=10 | lr=0.5 | epochs=15


  0%|          | 0/15 [00:00<?, ?it/s]

...

validation accuracy: 80.92
searching 11 / 12,  n_class=10 | lr=0.005 | epochs=10


  0%|          | 0/10 [00:00<?, ?it/s]

validation accuracy: 80.58999999999999
searching 12 / 12,  n_class=10 | lr=0.005 | epochs=15


  0%|          | 0/15 [00:00<?, ?it/s]

validation accuracy: 80.60000000000001
DONE. Best parameter is n_class=10 | lr=0.08 | epochs=15


In [162]:
lr = 0.08
n_epochs = 15

percept_fashion = Perceptron(n_class_fashion, lr, n_epochs)
percept_fashion.train(X_train_fashion, y_train_fashion)

  0%|          | 0/15 [00:00<?, ?it/s]

In [163]:
pred_percept = percept_fashion.predict(X_train_fashion)
print('The training accuracy is given by: %f' % (get_acc(pred_percept, y_train_fashion)))

The training accuracy is given by: 84.612000


### Validate Perceptron on Fashion-MNIST

In [164]:
pred_percept = percept_fashion.predict(X_val_fashion)
print('The validation accuracy is given by: %f' % (get_acc(pred_percept, y_val_fashion)))

The validation accuracy is given by: 81.700000


### Test Perceptron on Fashion-MNIST

In [165]:
pred_percept = percept_fashion.predict(X_test_fashion)
print('The testing accuracy is given by: %f' % (get_acc(pred_percept, y_test_fashion)))

The testing accuracy is given by: 81.000000


### Perceptron_Fashion-MNIST Kaggle Submission

Once you are satisfied with your solution and test accuracy, output a file to submit your test set predictions to the Kaggle for Assignment 1 Fashion-MNIST. Use the following code to do so:

In [ ]:
# output_submission_csv('kaggle/perceptron_submission_fashion.csv', percept_fashion.predict(X_test_fashion))

## Train Perceptron on Rice

In [166]:
lr = 0.08
n_epochs = 15

percept_RICE = Perceptron(n_class_RICE, lr, n_epochs)
percept_RICE.train(X_train_RICE, y_train_RICE)

  0%|          | 0/15 [00:00<?, ?it/s]

In [167]:
pred_percept = percept_RICE.predict(X_train_RICE)
print('The training accuracy is given by: %f' % (get_acc(pred_percept, y_train_RICE)))

The training accuracy is given by: 99.899184


### Validate Perceptron on Rice

In [168]:
pred_percept = percept_RICE.predict(X_val_RICE)
print('The validation accuracy is given by: %f' % (get_acc(pred_percept, y_val_RICE)))

The validation accuracy is given by: 99.835029


### Test Perceptron on Rice

In [169]:
pred_percept = percept_RICE.predict(X_test_RICE)
print('The testing accuracy is given by: %f' % (get_acc(pred_percept, y_test_RICE)))

The testing accuracy is given by: 99.835029


# Support Vector Machines (with SGD)

Next, you will implement a "soft margin" SVM. In this formulation you will maximize the margin between positive and negative training examples and penalize margin violations using a hinge loss.

We will optimize the SVM loss using SGD. This means you must compute the loss function with respect to model weights. You will use this gradient to update the model weights.

SVM optimized with SGD has 3 hyperparameters that you can experiment with:
- **Learning rate** - similar to as defined above in Perceptron, this parameter scales by how much the weights are changed according to the calculated gradient update. 
- **Epochs** - similar to as defined above in Perceptron.
- **Regularization constant** - Hyperparameter to determine the strength of regularization. In this case it is a coefficient on the term which maximizes the margin. You could try different values. The default value is set to 0.05.

You will implement the SVM using SGD in the **models/svm.py**

The following code: 
- Creates an instance of the SVM classifier class 
- The train function of the SVM class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

## Train SVM on Fashion-MNIST

In [171]:
params = {
    "model": SVM,
    "xtrain": X_train_fashion,
    "ytrain": y_train_fashion,
    "xval": X_test_fashion,
    "yval": y_test_fashion,
    "hyperparams": {
        "n_class": [n_class_fashion],
        "lr": [0.5, 0.1, 0.08, 0.05, 0.01, 0.005],
        "epochs": [10],
        "reg_const": [0.1, 0.05, 0.01]
    }
}
# search(params)

  0%|          | 0/18 [00:00<?, ?it/s]

searching 1 / 18,  n_class=10 | lr=0.5 | epochs=10 | reg_const=0.1
validation accuracy: 81.58999999999999
searching 2 / 18,  n_class=10 | lr=0.5 | epochs=10 | reg_const=0.05
validation accuracy: 81.58999999999999
searching 3 / 18,  n_class=10 | lr=0.5 | epochs=10 | reg_const=0.01
validation accuracy: 81.47999999999999
searching 4 / 18,  n_class=10 | lr=0.1 | epochs=10 | reg_const=0.1
validation accuracy: 81.47999999999999
searching 5 / 18,  n_class=10 | lr=0.1 | epochs=10 | reg_const=0.05
validation accuracy: 81.47999999999999
searching 6 / 18,  n_class=10 | lr=0.1 | epochs=10 | reg_const=0.01
validation accuracy: 81.69
searching 7 / 18,  n_class=10 | lr=0.08 | epochs=10 | reg_const=0.1
validation accuracy: 81.38
searching 8 / 18,  n_class=10 | lr=0.08 | epochs=10 | reg_const=0.05
validation accuracy: 81.55
searching 9 / 18,  n_class=10 | lr=0.08 | epochs=10 | reg_const=0.01
validation accuracy: 81.32000000000001
searching 10 / 18,  n_class=10 | lr=0.05 | epochs=10 | reg_const=0.1
vali

In [176]:
lr = 0.1
n_epochs = 10
reg_const = 0.01

svm_fashion = SVM(n_class_fashion, lr, n_epochs, reg_const)
svm_fashion.train(X_train_fashion, y_train_fashion)

In [177]:
pred_svm = svm_fashion.predict(X_train_fashion)
print('The training accuracy is given by: %f' % (get_acc(pred_svm, y_train_fashion)))

The training accuracy is given by: 84.634000


### Validate SVM on Fashion-MNIST

In [178]:
pred_svm = svm_fashion.predict(X_val_fashion)
print('The validation accuracy is given by: %f' % (get_acc(pred_svm, y_val_fashion)))

The validation accuracy is given by: 82.570000


### Test SVM on Fashion-MNIST

In [179]:
pred_svm = svm_fashion.predict(X_test_fashion)
print('The testing accuracy is given by: %f' % (get_acc(pred_svm, y_test_fashion)))

The testing accuracy is given by: 81.440000


### SVM_Fashion-MNIST Kaggle Submission

Once you are satisfied with your solution and test accuracy output a file to submit your test set predictions to the Kaggle for Assignment 1 Fashion-MNIST. Use the following code to do so:

In [180]:
# output_submission_csv('kaggle/svm_submission_fashion.csv', svm_fashion.predict(X_test_fashion))

## Train SVM on Rice

In [181]:
lr = 0.1
n_epochs = 10
reg_const = 0.01

svm_RICE = SVM(n_class_RICE, lr, n_epochs, reg_const)
svm_RICE.train(X_train_RICE, y_train_RICE)

In [182]:
pred_svm = svm_RICE.predict(X_train_RICE)
print('The training accuracy is given by: %f' % (get_acc(pred_svm, y_train_RICE)))

The training accuracy is given by: 97.937861


### Validate SVM on Rice

In [183]:
pred_svm = svm_RICE.predict(X_val_RICE)
print('The validation accuracy is given by: %f' % (get_acc(pred_svm, y_val_RICE)))

The validation accuracy is given by: 98.267803


## Test SVM on Rice

In [184]:
pred_svm = svm_RICE.predict(X_test_RICE)
print('The testing accuracy is given by: %f' % (get_acc(pred_svm, y_test_RICE)))

The testing accuracy is given by: 97.937861


# Softmax Classifier (with SGD)

Next, you will train a Softmax classifier. This classifier consists of a linear function of the input data followed by a softmax function which outputs a vector of dimension C (number of classes) for each data point. Each entry of the softmax output vector corresponds to a confidence in one of the C classes, and like a probability distribution, the entries of the output vector sum to 1. We use a cross-entropy loss on this sotmax output to train the model. 

Check the following link as an additional resource on softmax classification: http://cs231n.github.io/linear-classify/#softmax

Once again we will train the classifier with SGD. This means you need to compute the gradients of the softmax cross-entropy loss function according to the weights and update the weights using this gradient. Check the following link to help with implementing the gradient updates: https://deepnotes.io/softmax-crossentropy

The softmax classifier has 3 hyperparameters that you can experiment with:
- **Learning rate** - As above, this controls how much the model weights are updated with respect to their gradient.
- **Number of Epochs** - As described for perceptron.
- **Regularization constant** - Hyperparameter to determine the strength of regularization. In this case, we minimize the L2 norm of the model weights as regularization, so the regularization constant is a coefficient on the L2 norm in the combined cross-entropy and regularization objective.

You will implement a softmax classifier using SGD in the **models/softmax.py**

The following code: 
- Creates an instance of the Softmax classifier class 
- The train function of the Softmax class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

## Train Softmax on Fashion-MNIST

In [186]:
params = {
    "model": Softmax,
    "xtrain": X_train_fashion,
    "ytrain": y_train_fashion,
    "xval": X_test_fashion,
    "yval": y_test_fashion,
    "hyperparams": {
        "n_class": [n_class_fashion],
        "lr": [0.5, 0.1, 0.08, 0.05, 0.01, 0.005],
        "epochs": [15],
        "reg_const": [0.1, 0.05, 0.01]
    }
}
search(params)

  0%|          | 0/18 [00:00<?, ?it/s]

searching 1 / 18,  n_class=10 | lr=0.5 | epochs=15 | reg_const=0.1
validation accuracy: 82.07
searching 2 / 18,  n_class=10 | lr=0.5 | epochs=15 | reg_const=0.05
validation accuracy: 82.36
searching 3 / 18,  n_class=10 | lr=0.5 | epochs=15 | reg_const=0.01
validation accuracy: 82.03
searching 4 / 18,  n_class=10 | lr=0.1 | epochs=15 | reg_const=0.1
validation accuracy: 82.24000000000001
searching 5 / 18,  n_class=10 | lr=0.1 | epochs=15 | reg_const=0.05
validation accuracy: 81.92
searching 6 / 18,  n_class=10 | lr=0.1 | epochs=15 | reg_const=0.01
validation accuracy: 81.89
searching 7 / 18,  n_class=10 | lr=0.08 | epochs=15 | reg_const=0.1
validation accuracy: 82.17
searching 8 / 18,  n_class=10 | lr=0.08 | epochs=15 | reg_const=0.05
validation accuracy: 82.25
searching 9 / 18,  n_class=10 | lr=0.08 | epochs=15 | reg_const=0.01
validation accuracy: 82.09
searching 10 / 18,  n_class=10 | lr=0.05 | epochs=15 | reg_const=0.1
validation accuracy: 82.04
searching 11 / 18,  n_class=10 | lr=0

In [187]:
lr = 0.5
n_epochs = 15
reg_const = 0.05

softmax_fashion = Softmax(n_class_fashion, lr, n_epochs, reg_const)
softmax_fashion.train(X_train_fashion, y_train_fashion)

In [188]:
pred_softmax = softmax_fashion.predict(X_train_fashion)
print('The training accuracy is given by: %f' % (get_acc(pred_softmax, y_train_fashion)))

The training accuracy is given by: 85.268000


### Validate Softmax on Fashion-MNIST

In [189]:
pred_softmax = softmax_fashion.predict(X_val_fashion)
print('The validation accuracy is given by: %f' % (get_acc(pred_softmax, y_val_fashion)))

The validation accuracy is given by: 83.090000


### Testing Softmax on Fashion-MNIST

In [190]:
pred_softmax = softmax_fashion.predict(X_test_fashion)
print('The testing accuracy is given by: %f' % (get_acc(pred_softmax, y_test_fashion)))

The testing accuracy is given by: 82.260000


### Softmax_Fashion-MNIST Kaggle Submission

Once you are satisfied with your solution and test accuracy output a file to submit your test set predictions to the Kaggle for Assignment 1 Fashion-MNIST. Use the following code to do so:

In [204]:
# output_submission_csv('kaggle/softmax_submission_fashion.csv', softmax_fashion.predict(X_test_fashion))

## Train Softmax on Rice

In [199]:
params = {
    "model": Softmax,
    "xtrain": X_train_RICE,
    "ytrain": y_train_RICE,
    "xval": X_test_RICE,
    "yval": y_test_RICE,
    "hyperparams": {
        "n_class": [n_class_RICE],
        "lr": [0.5, 0.1, 0.08, 0.05, 0.01, 0.005],
        "epochs": [15],
        "reg_const": [0.1, 0.05, 0.01]
    }
}
search(params)

  0%|          | 0/18 [00:00<?, ?it/s]

searching 1 / 18,  n_class=2 | lr=0.5 | epochs=15 | reg_const=0.1
validation accuracy: 82.9529832279351
searching 2 / 18,  n_class=2 | lr=0.5 | epochs=15 | reg_const=0.05
validation accuracy: 85.56502612042893
searching 3 / 18,  n_class=2 | lr=0.5 | epochs=15 | reg_const=0.01
validation accuracy: 97.52543304921639
searching 4 / 18,  n_class=2 | lr=0.1 | epochs=15 | reg_const=0.1
validation accuracy: 91.64146274401979
searching 5 / 18,  n_class=2 | lr=0.1 | epochs=15 | reg_const=0.05
validation accuracy: 94.63843827330217
searching 6 / 18,  n_class=2 | lr=0.1 | epochs=15 | reg_const=0.01
validation accuracy: 93.42864998625241
searching 7 / 18,  n_class=2 | lr=0.08 | epochs=15 | reg_const=0.1
validation accuracy: 96.89304371734946
searching 8 / 18,  n_class=2 | lr=0.08 | epochs=15 | reg_const=0.05
validation accuracy: 92.38383282925487
searching 9 / 18,  n_class=2 | lr=0.08 | epochs=15 | reg_const=0.01
validation accuracy: 93.2911740445422
searching 10 / 18,  n_class=2 | lr=0.05 | epochs

In [200]:
lr = 0.01
n_epochs = 15
reg_const = 0.01

softmax_RICE = Softmax(n_class_RICE, lr, n_epochs, reg_const)
softmax_RICE.train(X_train_RICE, y_train_RICE)

In [201]:
pred_softmax = softmax_RICE.predict(X_train_RICE)
print('The training accuracy is given by: %f' % (get_acc(pred_softmax, y_train_RICE)))

The training accuracy is given by: 97.919531


### Validate Softmax on Rice

In [202]:
pred_softmax = softmax_RICE.predict(X_val_RICE)
print('The validation accuracy is given by: %f' % (get_acc(pred_softmax, y_val_RICE)))

The validation accuracy is given by: 98.240308


### Testing Softmax on Rice

In [203]:
pred_softmax = softmax_RICE.predict(X_test_RICE)
print('The testing accuracy is given by: %f' % (get_acc(pred_softmax, y_test_RICE)))

The testing accuracy is given by: 97.937861


# Logistic Classifier

The Logistic Classifier has 2 hyperparameters that you can experiment with:
- **Learning rate** - similar to as defined above in Perceptron, this parameter scales by how much the weights are changed according to the calculated gradient update. 
- **Number of Epochs** - As described for perceptron.
- **Threshold** - The decision boundary of the classifier.


You will implement the Logistic Classifier in the **models/logistic.py**

The following code: 
- Creates an instance of the Logistic classifier class 
- The train function of the Logistic class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

### Training Logistic Classifer

In [209]:
params = {
    "model": Logistic,
    "xtrain": X_train_RICE,
    "ytrain": y_train_RICE,
    "xval": X_test_RICE,
    "yval": y_test_RICE,
    "hyperparams": {
        "lr": [0.5, 0.1, 0.08, 0.05, 0.01, 0.005],
        "epochs": [15, 10],
        "threshold": [0.4,0.5,0.6,0.7,0.8]
    }
}
search(params)

  0%|          | 0/60 [00:00<?, ?it/s]

searching 1 / 60,  lr=0.5 | epochs=15 | threshold=0.4


D:\CODE\School\cs444\assignment1\models\logistic.py:30: RuntimeWarning: overflow encountered in exp
  


validation accuracy: 99.80753368160572
searching 2 / 60,  lr=0.5 | epochs=15 | threshold=0.5
validation accuracy: 99.80753368160572
searching 3 / 60,  lr=0.5 | epochs=15 | threshold=0.6
validation accuracy: 99.80753368160572
searching 4 / 60,  lr=0.5 | epochs=15 | threshold=0.7
validation accuracy: 99.83502886994776
searching 5 / 60,  lr=0.5 | epochs=15 | threshold=0.8
validation accuracy: 99.80753368160572
searching 6 / 60,  lr=0.5 | epochs=10 | threshold=0.4
validation accuracy: 99.80753368160572
searching 7 / 60,  lr=0.5 | epochs=10 | threshold=0.5
validation accuracy: 99.80753368160572
searching 8 / 60,  lr=0.5 | epochs=10 | threshold=0.6
validation accuracy: 99.80753368160572
searching 9 / 60,  lr=0.5 | epochs=10 | threshold=0.7
validation accuracy: 99.80753368160572
searching 10 / 60,  lr=0.5 | epochs=10 | threshold=0.8
validation accuracy: 99.80753368160572
searching 11 / 60,  lr=0.1 | epochs=15 | threshold=0.4
validation accuracy: 99.78003849326367
searching 12 / 60,  lr=0.1 | 

In [210]:
learning_rate = 0.005
n_epochs = 15
threshold = 0.4

lr = Logistic(learning_rate, n_epochs, threshold)
lr.train(X_train_RICE, y_train_RICE)

In [211]:
pred_lr = lr.predict(X_train_RICE)
print('The training accuracy is given by: %f' % (get_acc(pred_lr, y_train_RICE)))

The training accuracy is given by: 99.880854


### Validate Logistic Classifer

In [212]:
pred_lr = lr.predict(X_val_RICE)
print('The validation accuracy is given by: %f' % (get_acc(pred_lr, y_val_RICE)))

The validation accuracy is given by: 99.862524


### Test Logistic Classifier

In [213]:
pred_lr = lr.predict(X_test_RICE)
print('The testing accuracy is given by: %f' % (get_acc(pred_lr, y_test_RICE)))

The testing accuracy is given by: 99.807534
